# Gradio

## Chatbot 예제

In [8]:
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage, AIMessage
import gradio as gr

In [9]:
# ChatOllama 모델 초기화
model = ChatOllama(model="gemma2", temperature=0.7, verbose=False) # temperature가 커질수록 자유도가 올라간다(자유도가 없다 = 입력 그대로 출력)

In [10]:
# 채팅 기록을 포함하여 응답을 생성하는 함수
def chat(message, history):
    # 이전 대화 기록을 ChatOllama 형식으로 변환
    chat_history = []
    for human, ai in history:
        chat_history.append(HumanMessage(content=human))
        chat_history.append(AIMessage(content=ai))

    # 현재 메시지 추가
    chat_history.append(HumanMessage(content=message))

    # 모델을 사용하여 응답 생성
    response = model.invoke(chat_history)

    return response.content

In [11]:
# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=chat,
    examples=[
        "안녕하세요!",
        "인공지능에 대해 설명해주세요.",
        "파이썬의 장점은 무엇인가요?"
    ],
    title="AI 챗봇",
    description="질문을 입력하면 AI가 답변합니다."
)


/Users/jojungon/AI_Project/llm/lib/python3.10/site-packages/gradio/components/chatbot.py:279: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


In [13]:
# 서버 실행
demo.launch(server_port=7860, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


In [14]:
demo.close()

Closing server running on port: 7860


## Chatbot 예제(Gradio + csv)

In [15]:
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
import gradio as gr

In [16]:
# CSV 파일 로드
df = pd.read_csv("../dataset/indata_kor.csv", encoding='CP949')
df.tail()

,inputs,response
27,한국폴리텍대학 스마트금융과의 최종 아웃풋은 어떤건가요?,스마트금융과는 찍어내기식의 포트폴리오가 아니라 매년 업체에서 요구하는 기술 및 주제...
28,한국폴리텍대학 스마트금융과의 최종 포트폴리오는 어떤건가요?,유튜브 채널에서 스마트금융과를 검색하시면 한국폴리텍대학 스마트금융과 포트폴리오 발표...
29,한국폴리텍대학 스마트금융과 면접시에는 어떤걸 준비하고 가면 될까요?,영문 타자연습 및 스마트금융과에 대한 열정을 보여주면 좋다
30,한국폴리텍대학 스마트금융과 입학 전까지 어떤걸 공부하면 될까요?,기본적인 OA를 잘 다루고 기본코드는 HKCODE의 기본 내용은 보고오면 됨. 파이...
31,한국폴리텍대학 스마트금융과는 대면/비대면 수업 어떻게 진행되나요?,대면으로 진행합니다.


In [17]:
# 텍스트 분할
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text("\n".join(df.to_string()))

In [19]:

# 임베딩 모델 초기화
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/var/folders/t5/w4syjrms3gn00d04kqv8dbyr0000gn/T/ipykernel_76771/2819514520.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [20]:
# 벡터 데이터베이스 생성
vectorstore = FAISS.from_texts(texts, embeddings)

In [21]:
# ChatOllama 모델 초기화
llm = ChatOllama(model="gemma2", tempeature=0.1)

/var/folders/t5/w4syjrms3gn00d04kqv8dbyr0000gn/T/ipykernel_76771/3071692779.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="gemma2", tempeature=0.1)


In [22]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(search_kwargs={"k":1}),
    return_source_documents=True,
    verbose=False
)

In [23]:
# 채팅 함수 정의
def chat(message, history):
    # 이전 대화 기록을 ConversationalRetrievalChain 형식으로 변환
    chat_history = [(human, ai) for human, ai in history]

    # 모델을 사용하여 응답 생성
    response = qa_chain({"question": message, "chat_history": chat_history})

    # 소스 문서 정보 추출
    sources = set([doc.metadata.get('source', 'Unknown') for doc in response['source_documents']])
    source_info = f"\n\n참고 출처: {', '.join(sources)}" if sources else ""

    return response['answer'] + source_info

In [24]:
# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=chat,
    examples=[
        "한국폴리텍대학 스마트금융과 면접시에는 어떤걸 준비하고 가면 될까요?",
        "스마트금융과에 대해 설명해주세요",
        "한국폴리텍대한 추천할만한 학과 하나를 소개해주세요."
    ],
    title="대학 정보 AI 챗봇",
    description="스마트금융과에 대한 질문을 입력하면 AI가 CSV데이터를 참고하여 한글로 답변합니다."
)

/Users/jojungon/AI_Project/llm/lib/python3.10/site-packages/gradio/components/chatbot.py:279: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


In [25]:
# 서버 실행
demo.launch(server_port=7862, server_name="0.0.0.0")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on local URL:  http://0.0.0.0:7862

To create a public link, set `share=True` in `launch()`.


/var/folders/t5/w4syjrms3gn00d04kqv8dbyr0000gn/T/ipykernel_76771/4160252103.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"question": message, "chat_history": chat_history})


In [26]:
demo.close()

Closing server running on port: 7862


## Chatbot 예제(인터넷 URL정보 요약)

In [17]:
import gradio as gr
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
import ollama
import logging

In [18]:
# Function to load, split, and retrieve documents

def load_and_retrieve_docs(url):
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs=dict()
    )

    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    embeddings = OllamaEmbeddings(model="gemma2")

    # vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
    vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

    return vectorstore.as_retriever()

In [19]:
# Function to format documents

def format_docs(docs):

    return "\n\n".join(doc.page_content for doc in docs)

In [20]:
def rag_chain(url, question):
    try: 
        retriever = load_and_retrieve_docs(url)

        retrieved_docs = retriever.invoke(question)

        # 검색된 문서가 비어있는지 확인
        if not retrieved_docs:
            raise ValueError("No documents retrieved. Please check the retrieval process.")

        # 문서 형식 지정
        formatted_context = format_docs(retrieved_docs)

        # # 만약 formatted_context가 너무 길면 트림 처리 등을 추가할 수 있음
        # if len(formatted_context) > 1000:
        #     formatted_context = formatted_context[:1000]  # 예시로 1000자 이내로 자르기

        # 질문과 문맥을 합쳐서 프롬프트 생성
        formatted_prompt = f"Question: {question}\n\nContext: {formatted_context}"

        # 요청 본문에 'data' 필드를 추가하여 API 호출
        response = ollama.chat(model='gemma2', messages=[{
            'role': 'user', 
            'content': formatted_prompt,
            'data': {'model': 'gemma2', 'prompt': formatted_prompt}  # 'data' 필드를 추가
        }])

        # 응답이 정상적인지 확인
        if 'message' not in response or 'content' not in response['message']:
            raise ValueError("Invalid response from model. 'message' or 'content' missing.")
        
        # 모델 응답 반환
        return response['message']['content']

    except Exception as e:
        # 오류가 발생하면 로그로 출력하고 None 반환
        logging.error(f"Error in rag_chain: {e}")
        return None


In [21]:
# # Gradio interface

# iface = gr.Interface(

#     fn=rag_chain,

#     inputs=["text", "text"],

#     outputs="text",

#     title="RAG Chain Question Answering",

#     description="Enter a URL and a query to get answers from the RAG chain."

# )

In [22]:
# Gradio Tabbed Interface
with gr.Blocks() as iface:
    # Tab for Question and Answer
    with gr.Tab("질문과 답변"):
        gr.Interface(
            fn=rag_chain,
            inputs=["text", "text"],
            outputs="text",
            title="RAG Chain Question Answering",
            description="Enter a URL and a query to get answers from the RAG chain."
        ).render()

    # Tab for Visualization (Word Cloud)
    with gr.Tab("시각화 (워드클라우드)"):
        gr.Markdown("이 탭은 시각화를 위한 공간입니다. 워드클라우드 기능이 여기에 추가될 예정입니다.")

In [24]:
# 디버그 모드로 Gradio 인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----


ValueError: When localhost is not accessible, a shareable link must be created. Please set share=True or check your proxy settings to allow access to localhost.

In [25]:
iface.close()

Closing server running on port: 7861


## 챗봇 예제(SST:음성을 텍스트로 전환)

In [37]:
# 사전 설치 : pip install openai-whisper
import os
from dotenv import load_dotenv
import whisper #type: ignore
import gradio as gr

In [38]:
# .env 파일에서 환경변수 로드(필요한 경우)
load_dotenv()

False

In [39]:
# ffmpeg 경로 명시적 설정
# os.envrion["FFMPEG_BINARY"] = "C:/aiproject/ffmpeg/bin/ffmpeg.exe"
os.environ["PATH"] += os.pathsep + r"/opt/homebrew/bin"
os.environ["FFMPEG_BINARY"] = r"/opt/homebrew/bin/ffmpeg"

In [40]:
def transcribe_audio(audio_path):
    # Whisper 모델 로드
    model = whisper.load_model("base")

    # 오디오 파일 전사
    result = model.transcribe(audio_path)

    # 전사된 텍스트 반환
    return result["text"]

In [41]:
def process_audio(audio):
    if audio is None:
        return "오디오 파일을 업로드 해주세요."
    try:
        transcribe_text = transcribe_audio(audio)
        return transcribe_text
    except Exception as e:
        return  f"오류가 발생했습니다: {str(e)}"

In [42]:
# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath", label="MP3 파일 업로드"),
    outputs="text",
    title = "MP3 to Text Converter",
    description="MP3 파일을 업로드하면 텍스트로 변환합니다."
)

In [43]:
# 디버그 모드로 Gradio 인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [44]:
iface.close()

Closing server running on port: 7861


## Chatbot 예제(TTS: 텍스트를 음성변환)

In [6]:
# 사전 설치 : pip install gtts
import gradio as gr
from gtts import gTTS #type: ignore
import os
import tempfile

In [7]:
def text_to_speech(text, lang="ko"):
    # 임시 파일 생성
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
        temp_filename = fp.name

    # TTS 변환
    tts = gTTS(text=text, lang=lang)
    tts.save(temp_filename)
    return temp_filename

In [8]:
def process_tts(text, lang):
    if not text:
        return None, "텍스트를 입력해주세요."
    try:
        audio_file = text_to_speech(text, lang)
        return audio_file, "변환이 완료되었습니다. 아래에서 재생 또는 다운로드 할 수 있습니다."
    except Exception as e:
        return None, f"오류가 발생했습니다: {str(e)}"

In [9]:
# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=process_tts,
    inputs=[
        gr.Textbox(lines=5, label="텍스트 입력"),
        gr.Dropdown(choices=['ko', 'en', 'ja', 'zh-cn'], label="언어 선택", value='ko')
    ],
    outputs=[
        gr.Audio(label="생성된 오디오"),
        gr.Textbox(label="상태 메시지")
    ],
    title = "Text to Speech Converter",
    description="텍스트를 입력하연 MP3 파일로 변환합니다."
)

In [10]:
# 디버그 모드로 Gradio 인터페이스 실행
iface.launch(server_port=11434, server_name="0.0.0.0", debug=True)

* Running on local URL:  http://0.0.0.0:11434

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [11]:
iface.close()

Closing server running on port: 11434


## 이미지 분류 예제(Gradio 사용)

In [63]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
from io import BytesIO

In [64]:
# TensorFlow MobileNetV2 모델 로드
model = tf.keras.applications.MobileNetV2(weights="imagenet")

In [ ]:
def predict_image(image_url):
    try:
        # URL에서 이미지 가져오기
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).resize((224, 224)) # BytesIO 사용하여 이미지 열기

        # 이미지를 배열로 변환
        image_array = tf.keras.preprocessing.image.img_to_array(image)
        image_array = tf.expand_dims(image_array, axis=0) # 배치 차원 추가
        image_array = tf.keras.applications.mobilenet_v2.preprocess_input(image_array) # 전처리

        # 예측 수행
        predictions = model.predict(image_array)
        decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predcitions(predictions, top=3)[0] # 상위 3개 예측

        # Gradio Label 컴포넌트에 맞게 결과 형식 변경
        # {label1: confidence1, label2: confidence2, ...} 형식으로 반환
        result = {label: float(prob) for (_, label, prob) in decoded_predictions}
        return result

    except Exception as e:
        return {"error": 1.0} # 에러 발생 시 기본값 반환

In [ ]:
# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Textbox(label="이미지 URL 입력"),
    outputs=gr.Label(num_top_classes=3, label="예측 결과"),
    title="음식 이미지 분류",
    description="이미지 URL을 입력하면 상위 3개의 예측 결과를 확률과 함께 표시합니다."
)

In [ ]:
# 인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

# 예시 이미지 URL : https://health.chosun.com/site/data/img_dir/2024/04/19/2024041901914_0.jpg

In [ ]:
iface.close()

## 이미지 분류 예제(Gradio + gemma2 사용)

In [55]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
from io import BytesIO
from langchain_community.chat_models import ChatOllama

In [56]:
# TensorFlow MobileNetV2 모델 로드
model = tf.keras.applications.MobileNetV2(weights="imagenet")

In [57]:
OLLAMA_SERVER = "http://localhost:11434"  # 로컬 서버 주소
MODEL_NAME = "gemma2"  # 사용하려는 Ollama 모델 이름

In [58]:
# Ollama를 사용해 음식 설명 생성
def get_food_description_with_langchain(food_name):
    """
    LangChain ChatOllama를 사용하여 음식 설명 생성
    """
    try:
        chat = ChatOllama(base_url=OLLAMA_SERVER, model=MODEL_NAME)
        prompt = f"Tell me about {food_name}."
        response = chat.predict(prompt)
        return response
    except Exception as e:
        return f"Failed to retrieve description: {e}"

In [59]:
# 이미지 예측 함수
def predict_image_with_description(image_url):
    """
    이미지 URL을 받아 음식 예측과 Ollama 설명을 반환
    """
    try:
        # URL에서 이미지 가져오기
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).resize((224, 224))  # BytesIO 사용하여 이미지 열기

        # 이미지를 배열로 변환
        image_array = tf.keras.preprocessing.image.img_to_array(image)
        image_array = tf.expand_dims(image_array, axis=0)  # 배치 차원 추가
        image_array = tf.keras.applications.mobilenet_v2.preprocess_input(image_array)  # 전처리

        # 예측 수행
        predictions = model.predict(image_array)
        decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=3)[0]  # 상위 3개 예측 결과 반환

        # 예측 결과 형식화
        result = {label: float(prob) for (_, label, prob) in decoded_predictions}

        # 가장 높은 확률의 예측값으로 Ollama 설명 생성
        top_food = decoded_predictions[0][1]  # 가장 확률이 높은 음식 이름
        description = get_food_description_with_langchain(top_food)

        return result, description  # 예측 결과와 Ollama 설명 반환

    except Exception as e:
        return {"error": 1.0}, f"Error: {e}"  # 에러 발생 시 기본값 반환

In [60]:
# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=predict_image_with_description,
    inputs=gr.Textbox(label="이미지 URL 입력"),
    outputs=[
        gr.Label(num_top_classes=3, label="예측 결과"),  # 상위 3개 예측 결과
        gr.Textbox(label="음식 설명", interactive=False)  # Ollama로 생성한 설명 출력
    ],
    title="음식 이미지 분류 및 설명 생성기",
    description="이미지 URL을 입력하면 음식 분류 결과와 설명을 제공합니다."
)

In [61]:
# 인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/var/folders/t5/w4syjrms3gn00d04kqv8dbyr0000gn/T/ipykernel_97705/2595182604.py:7: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  chat = ChatOllama(base_url=OLLAMA_SERVER, model=MODEL_NAME)
/var/folders/t5/w4syjrms3gn00d04kqv8dbyr0000gn/T/ipykernel_97705/2595182604.py:9: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat.predict(prompt)


Keyboard interruption in main thread... closing server.


In [62]:
iface.close()

Closing server running on port: 7861


## 기타(워드클라우드 gradio 예제)

In [10]:
import gradio as gr
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [11]:
# 워드 클라우드 생성함수
def generate_wordcloud(file_obj):
    try:
        # 파일이 없는 경우 처리
        if file_obj is None:
            return None
        
        # Gradio의 파일 객체에서 파일 경로 가져오기
        file_path = file_obj.name
        
        # 파일 읽기
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()

        # 워드클라우드 생성
        wordcloud = WordCloud(
            font_path="NanumGothic", # 한글 폰트 설정
            background_color="white",
            width=800,
            height=600,
            max_words=200,
            max_font_size=100,
            min_font_size=10,
            random_state=42
        ).generate(text)
        
        # 워드클라우드 이미지 플롯
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation="bilinear")
        plt.axis("off")
        plt.tight_layout()

        # 결과 이미지 저장
        output_path = "wordcloud.png"
        plt.savefig(output_path)
        plt.close() # 메모리 누수 방지를 위해 figure 닫기
        return output_path
    
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

In [12]:
# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=generate_wordcloud,
    inputs=gr.File(label="Upload a .txt file"),
    outputs=gr.Image(type="filepath", label="Word Cloud")
)

In [13]:
iface.launch(server_port=7861, share=True, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7861
* Running on public URL: https://051312234ca73f08fc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [40]:
iface.close()

Closing server running on port: 7861


## 자기소개서 도우미 챗봇 예제(Gradio)

In [15]:
# 사전 설치 : pip install
import os
import gradio as gr
from langchain.llms import Ollama
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from fpdf import FPDF

In [30]:
# Ollama 설정 (Gemma2 모델 사용)
os.environ["OLLAMA_API_BASE"] = "http://localhost:11434"  # Ollama 서버 주소
ollama_model = Ollama(model="gemma2")

/var/folders/t5/w4syjrms3gn00d04kqv8dbyr0000gn/T/ipykernel_25041/2509825096.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  ollama_model = Ollama(model="gemma2")


In [31]:
# 다양한 템플릿 설정
TEMPLATES = {
    "취업": "Based on the following keywords and example, write a personal statement for a job application.",
    "대학원": "Using the provided keywords, draft a personal statement for a graduate school application.",
    "봉사활동": "With the given keywords, write a personal statement emphasizing volunteer experience and motivation."
}

In [32]:
# 언어 지원: 한국어, 영어, 일본어
LANGUAGES = {
    "한국어": "Please write the response in Korean.",
    "영어": "Please write the response in English.",
    "일본어": "Please write the response in Japanese."
}

In [33]:
# 자동 키워드 추천 함수
def recommend_keywords(purpose):
    if purpose == "취업":
        return "책임감, 팀워크, 문제 해결 능력"
    elif purpose == "대학원":
        return "연구 열정, 창의력, 학업 성취도"
    elif purpose == "봉사활동":
        return "사회적 책임감, 희생정신, 리더십"
    else:
        return ""

In [34]:
# 자기소개서 작성 함수
def generate_statement(purpose, language, keywords, example_sentence=None):
    if purpose not in TEMPLATES:
        return "❌ 지원 목적을 올바르게 선택해주세요."
    if language not in LANGUAGES:
        return "❌ 언어를 올바르게 선택해주세요."

    # 템플릿 생성
    template = TEMPLATES[purpose] + "\n\nKeywords: {keywords}\n" + LANGUAGES[language]
    if example_sentence:
        template += f"\n\nExample sentence: {example_sentence}"

    prompt = PromptTemplate(input_variables=["keywords"], template=template)
    chain = LLMChain(llm=ollama_model, prompt=prompt)
    response = chain.run({"keywords": keywords})
    return response

In [35]:
# PDF 저장 함수
def save_to_pdf(statement, filename="personal_statement.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.add_font('NanumGothic', '', r'C:\Windows\Fonts\malgun.ttf', uni=True)  # '맑은 고딕' 폰트 경로 설정
    pdf.set_font('NanumGothic', size=12)  # 폰트 설정
    pdf.multi_cell(0, 10, statement)
    pdf.output(filename)
    return f"✔️ PDF 저장 완료: {filename}"

In [36]:
# Gradio 인터페이스
def chatbot_interface(purpose, language, keywords, example_sentence=None, save_pdf=False):
    statement = generate_statement(purpose, language, keywords, example_sentence)
    if save_pdf:
        save_to_pdf(statement)
    return statement

In [37]:
with gr.Blocks() as demo:
    gr.Markdown("# 📝 다목적 자기소개서 작성 도우미")
    gr.Markdown("키워드와 추천 문장을 활용하여 취업, 대학원, 봉사활동 자기소개서를 생성하고 PDF로 저장하세요!")

    # 입력 영역
    with gr.Row():
        purpose_input = gr.Dropdown(label="지원 목적", choices=["취업", "대학원", "봉사활동"], value="취업")
        language_input = gr.Dropdown(label="언어 선택", choices=["한국어", "영어", "일본어"], value="한국어")

    recommended_keywords = gr.Textbox(label="추천 키워드", interactive=False)
    recommend_btn = gr.Button("키워드 추천")
    recommend_btn.click(recommend_keywords, inputs=[purpose_input], outputs=[recommended_keywords])

    with gr.Row():
        keywords_input = gr.Textbox(label="사용자 키워드 입력", placeholder="예: 책임감, 팀워크, 문제 해결 능력")
        example_sentence_input = gr.Textbox(
            label="추천 문장 (선택 사항)",
            placeholder="예: '저는 도전을 두려워하지 않고 성공적으로 프로젝트를 완수했습니다.'"
        )

    save_pdf_toggle = gr.Checkbox(label="PDF로 저장", value=False)

    # 출력 영역
    output = gr.Textbox(label="작성된 자기소개서", lines=6)
    submit_btn = gr.Button("작성하기")
    submit_btn.click(
        fn=chatbot_interface,
        inputs=[purpose_input, language_input, keywords_input, example_sentence_input, save_pdf_toggle],
        outputs=[output]
    )

In [38]:
# 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


/var/folders/t5/w4syjrms3gn00d04kqv8dbyr0000gn/T/ipykernel_25041/4139684956.py:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=ollama_model, prompt=prompt)
/var/folders/t5/w4syjrms3gn00d04kqv8dbyr0000gn/T/ipykernel_25041/4139684956.py:15: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"keywords": keywords})


In [39]:
# 종료(자원회수)
demo.close()

Closing server running on port: 7862
